In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
import string
os.getcwd()

# Path to data file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\Spatial Join\\2018-08-23_fts_20secMed_QAQC_ebs_120619_spatialJoin.csv")

df_data = pd.read_csv(data_path)
df_data.head()


,FTS Timestamp,Latitude,Longitude,WS fDOM QSU,WS fCHLA ug/L,SUNA NO3 uM,UCI Timestamp,Processed NO3(uMol),CStar Tr,CStar c,...,EXO DO mg L,EXO DO sat,EXO Turb FNU,EXO fDOM QSU,EXO fCHLA ug/L,EXO fBGAPC ug L,index_right,Poly_ID_01,Poly_ID_5,Poly_ID_1
0,2018-08-23 08:49:43,38.169900,-121.674155,21.573216,1.989287,4.755,NaN,NaN,0.664697,4.084245,...,8.340,94.140,4.330,18.810668,0.260341,0.11,1738.0,A666,C14,B67
1,2018-08-23 08:49:44,38.169890,-121.674150,21.573568,1.987489,4.750,NaN,NaN,0.664816,4.082456,...,8.340,94.160,4.320,18.794815,0.260341,0.11,1738.0,A666,C14,B67
2,2018-08-23 08:49:45,38.169890,-121.674150,21.573216,1.985836,4.755,NaN,NaN,0.665274,4.075569,...,8.345,94.170,4.305,18.778682,0.255520,0.11,1738.0,A666,C14,B67
3,2018-08-23 08:49:46,38.169880,-121.674130,21.572864,1.984182,4.750,NaN,NaN,0.665732,4.068682,...,8.350,94.180,4.290,18.762550,0.250699,0.11,1738.0,A666,C14,B67
4,2018-08-23 08:49:47,38.169875,-121.674120,21.566991,1.981550,4.755,NaN,NaN,0.665924,4.065797,...,8.350,94.195,4.280,18.753300,0.236236,0.11,1738.0,A666,C14,B67


In [10]:
# Lets see what columns we have
df_data.columns.values

array(['FTS Timestamp', 'Latitude', 'Longitude', 'WS fDOM QSU',
       'WS fCHLA ug/L', 'SUNA NO3 uM', 'UCI Timestamp',
       'Processed NO3(uMol)', 'CStar Tr', 'CStar c', 'TSG Temp',
       'TSG Cond', 'TSG Salinity', 'EXO Temp', 'EXO Sp Cond', 'EXO pH',
       'EXO pH mV', 'EXO DO mg L', 'EXO DO sat', 'EXO Turb FNU',
       'EXO fDOM QSU', 'EXO fCHLA ug/L', 'EXO fBGAPC ug L', 'index_right',
       'Poly_ID_01', 'Poly_ID_5', 'Poly_ID_1'], dtype=object)

In [11]:
# Make a variable for the polygon id column name
# Change this to the name you identified in the list above
poly_id_col = "Poly_ID_01"

# Remove nulls in "Poly_ID column"
print(len(df_data))
df_data = df_data.dropna(subset=[poly_id_01_col])
print(len(df_data))


26241
25876


In [13]:
# Now lets make a list of the ones we don't want to include
# I am just copy/pasting non-constituent columns from above into this list
cols_to_drop = [
    'FTS Timestamp', 'Latitude', 'Longitude', 'index_right', 'UCI Timestamp']

df_data = df_data.drop(cols_to_drop, axis="columns")
df_data.columns.values

array(['WS fDOM QSU', 'WS fCHLA ug/L', 'SUNA NO3 uM',
       'Processed NO3(uMol)', 'CStar Tr', 'CStar c', 'TSG Temp',
       'TSG Cond', 'TSG Salinity', 'EXO Temp', 'EXO Sp Cond', 'EXO pH',
       'EXO pH mV', 'EXO DO mg L', 'EXO DO sat', 'EXO Turb FNU',
       'EXO fDOM QSU', 'EXO fCHLA ug/L', 'EXO fBGAPC ug L', 'Poly_ID_01',
       'Poly_ID_5', 'Poly_ID_1'], dtype=object)

In [18]:
# Compute Stats
# First group the data by poly_id_col
df_grouped = df_data.groupby(["Poly_ID_01", "Poly_ID_1", "Poly_ID_5"])

# Now lets define the statistics we want to compute in a list that we can pass to the pandas aggregation function
# For more information on what can go into this list check out: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
#KO: added std to stats list 5/19/2020

stats = [
    "min",
    "max",
    "mean",
    "median",
    "std"
]

# Compute the statistics defined above for each polygon
df_stats = df_grouped.agg(stats)

# Flatten the hierarchical columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

df_stats.head()


,,,WS fDOM QSU min,WS fDOM QSU max,WS fDOM QSU mean,WS fDOM QSU median,WS fDOM QSU std,WS fCHLA ug/L min,WS fCHLA ug/L max,WS fCHLA ug/L mean,WS fCHLA ug/L median,WS fCHLA ug/L std,...,EXO fCHLA ug/L min,EXO fCHLA ug/L max,EXO fCHLA ug/L mean,EXO fCHLA ug/L median,EXO fCHLA ug/L std,EXO fBGAPC ug L min,EXO fBGAPC ug L max,EXO fBGAPC ug L mean,EXO fBGAPC ug L median,EXO fBGAPC ug L std
Poly_ID_01,Poly_ID_1,Poly_ID_5,,,,,,,,,,,,,,,,,,,,,
A1971,B196,C46,25.111473,25.486354,25.252082,25.182367,0.122197,5.203949,5.928905,5.581667,5.680690,0.238138,...,1.189956,1.562044,1.421185,1.479031,0.117109,0.25,0.290,0.267887,0.2700,0.010690
A1972,B196,C46,25.104558,25.436674,25.252849,25.250074,0.134483,5.155590,5.864281,5.485510,5.369775,0.311031,...,1.170699,1.474166,1.276894,1.227354,0.102381,0.23,0.270,0.252632,0.2575,0.013035
A1973,B196,C46,25.172762,25.327386,25.247360,25.208072,0.060869,5.135600,5.475739,5.231843,5.184100,0.103806,...,1.146522,1.245076,1.194163,1.201187,0.035197,0.23,0.255,0.242097,0.2500,0.010549
A1974,B196,C46,25.199575,25.373390,25.268489,25.206411,0.075131,5.023007,5.149893,5.113207,5.134629,0.038473,...,1.136805,1.191460,1.161880,1.171894,0.021620,0.23,0.250,0.239688,0.2400,0.008418
A1975,B196,C46,25.192853,25.337474,25.246212,25.240258,0.042635,4.864060,5.131899,5.003180,5.002344,0.098693,...,1.040451,1.214477,1.158403,1.166975,0.043139,0.22,0.270,0.242414,0.2300,0.019938


In [20]:
# run the loop desired number of times. final output file will have the stats based on the final loop. 
# adjust number of iterations to make sure values converge using printed stat below
num_iterations = 12
tmp_df_data = df_data.copy() #copy of original df_data to preserve data
tmp_df_stats = df_stats.copy() 
for num in range(num_iterations):
    
    # merge original data and stats dataframes on poly ID
    df_merged = df_data.merge(tmp_df_stats, how = 'outer', left_on = 'Poly_ID_01', right_on = 'Poly_ID_01')
    
    convergence = 0
    for col in df_data.columns:
        # skip location, poly_id, lat, long columns when filtering data
        if col in ('FTS Timestamp', 'Latitude', 'Longitude', 'index_right', 'UCI Timestamp', 'Poly_ID_01', 'Poly_ID_1', 'Poly_ID_5'):
            continue
 
        # creates series to hold mean and standard deviation values
        col_mean = col + ' mean' 
        col_std = col + ' std' # standard deviation

        # calculates and creates a column for 2x std
        col_2xstd = 2*df_merged[col_std]

        # calculates the difference between the instantaneous value and the mean value of its corresponding polygon
        col_diff_mean = abs(df_merged[col] - df_merged[col_mean])

        # if the difference between the instantenous value and the mean is greater than 2xstd replace the value with a NAN
        remove_mask = col_diff_mean > col_2xstd
        df_merged[col][remove_mask] = np.nan   
        convergence += col_2xstd.sum()
    print("Sum of standard deviations converging on: ", convergence)

    # creates dataframe, df_data_filtered
    # dropping extra columns created during filtering
    # filtered data retained, and columns kept are based on columns that existed in df_data
    tmp_df_data = df_merged[df_data.columns]
    
    # Re-run stats on filtered data
    df_grouped_filtered = tmp_df_data.groupby(["Poly_ID_01", "Poly_ID_1", "Poly_ID_5"])

    # Compute the statistics defined above for each polygon
    tmp_df_stats = df_grouped_filtered.agg(stats)
    
    # Flatten the hierarchical columns
    tmp_df_stats.columns = [' '.join(col).strip() for col in tmp_df_stats.columns.values]

# change name of dataframe for clarity after processing
df_stats_filtered = tmp_df_stats

tmp_df_stats

# stats for each polygon based on final iteration of above loop
# Write output file
out_dir = "C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\Statistics"
out_fname = data_path.name.split(".")[0] + "_statsTest.csv"
out_path = Path(out_dir, out_fname)

# Flatten the hierarchical columns
df_stats_filtered.columns = [''.join(col).strip() for col in df_stats_filtered.columns.values]


# Write the csv
df_stats_filtered.to_csv(out_path)

C:\Users\jsoto\Anaconda3\envs\rmp-spatial-join-stats\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Sum of standard deviations converging on:  813454.7279051931
Sum of standard deviations converging on:  718223.9279707769
Sum of standard deviations converging on:  690186.3337507612
Sum of standard deviations converging on:  675974.2723238434
Sum of standard deviations converging on:  669820.0963102308
Sum of standard deviations converging on:  666024.8616835749
Sum of standard deviations converging on:  663910.9134446593
Sum of standard deviations converging on:  662695.4852388576
Sum of standard deviations converging on:  661746.6936432264
Sum of standard deviations converging on:  660733.7693987049
Sum of standard deviations converging on:  660244.0472059425
Sum of standard deviations converging on:  660061.8552914241


In [24]:
# use a left join to append lat, long values of delta polygon center points
# Path to center point file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_point01_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_01.xlsx")
data_point1_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_1.xlsx")
data_point5_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_5.xlsx")

df_point01 = pd.read_excel(data_point01_path)
df_point1 = pd.read_excel(data_point1_path)
df_point5 = pd.read_excel(data_point5_path)

df_join01 = pd.merge(left=df_stats_filtered, right=df_point01, how='left', left_on='Poly_ID_01', right_on='CL_ID')
df_join1 = pd.merge(left=df_stats_filtered, right=df_point1, how='left', left_on='Poly_ID_1', right_on='CL_ID')
df_join5 = pd.merge(left=df_stats_filtered, right=df_point5, how='left', left_on='Poly_ID_5', right_on='CL_ID')

# stats for each polygon based on final iteration of above loop joined with CL_ID
# Write output file
out_dir2 = "C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\CL_ID Join"
out_fname01 = data_path.name.split(".")[0] + "_statsTest_join01.csv"
out_path01 = Path(out_dir2, out_fname01)

out_fname1 = data_path.name.split(".")[0] + "_statsTest_join1.csv"
out_path1 = Path(out_dir2, out_fname1)

out_fname5 = data_path.name.split(".")[0] + "_statsTest_join5.csv"
out_path5 = Path(out_dir2, out_fname5)

# Flatten the hierarchical columns
df_join01.columns = [''.join(col).strip() for col in df_join01.columns.values]
df_join1.columns = [''.join(col).strip() for col in df_join1.columns.values]
df_join5.columns = [''.join(col).strip() for col in df_join5.columns.values]

# Write the csv
df_join01.to_csv(out_path01)
df_join1.to_csv(out_path1)
df_join5.to_csv(out_path5)